My first idea for this challenge is to create a subtracted image at each interval, and plug that into a CNN that in turn regresses onto the speed of the car.

In [9]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
%matplotlib inline

In [3]:
vidcap = cv2.VideoCapture('data/train.mp4')
train_speeds = np.loadtxt('data/train.txt')
sec = 0
frameRate = 1/20 #//it will capture image in each 0.5 second
count=0
#success = getFrame(sec)
imgs = []
img_diff = []
hasFrames = True
while count < 2040:
 #    print(count)
    count = count + 1
    sec = sec + frameRate
   # sec = round(sec, 2)
  #  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    imgs.append(image)
    if count == 0:
        #what should the first value be? 
        print("Skipped the first frame, and will do the same in the training set. ")
    if count > 1:
        img_diff.append(imgs[count-1] - imgs[count - 2])

        
#img diff as a pytorch tensor??

In [4]:
img_diff = np.array(img_diff)
train_speeds = train_speeds[1:len(img_diff)]

Data loader... Batch size needs to be designated now too!

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_val,y_train,y_val = train_test_split(img_diff,train_speeds)

In [41]:
batch_size = 16
train_loader = []
x = 0
while x < len(X_train) // batch_size:
    Xtrain_batch = []
    ytrain_batch = []
    i = 0
    while i < batch_size:
        Xtrain_batch.append(X_train[i])
        ytrain_batch.append(y_train[i])
        i +=1
        
    train_loader.append((torch.Tensor(Xtrain_batch).resize(batch_size,3,480,640),torch.Tensor(ytrain_batch)))
    X_train = X_train[i:]
    y_train = y_train[i:]
    x+=1

/Users/noahkasmanoff/anaconda3/lib/python3.7/site-packages/torch/tensor.py:330: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


KeyboardInterrupt: 

In [ ]:
batch_size = 16
val_loader = []
x = 0
while x < len(X_val) // batch_size:
    Xval_batch = []
    yval_batch = []
    i = 0
    while i < batch_size:
        Xval_batch.append(X_val[i])
        yval_batch.append(y_val[i])
        i +=1
        
    val_loader.append((torch.Tensor(Xval_batch).resize(batch_size,3,480,640),torch.Tensor(yval_batch)))
    X_val = X_val[i:]
    y_val = y_val[i:]
    x+=1

Now import pytorch, and prepare a model! 

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        ) 
        
        self.out = nn.Linear(32 * 120 * 160, 1)   # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        print("Shape after conv 2 ", x.shape)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output  

In [25]:
# define the loss and optimizer
#instantiate the model and quickly state how many parameters it has 
model =  CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print('Total number of parameters in the network: %d'%total_params)

Total number of parameters in the network: 628449


In [14]:
# define the loss and optimizer
criterion = nn.MSELoss()  #mean squared error is good for regression :)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, 
                             weight_decay=weight_decay)


########################################################################################

# create the training, validation and testing datasets


print("data is loaded in ! ")



#for weight decay 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.3, 
                                                       patience=10, verbose=True)


model.eval()   #what's this do 

for epoch in range(epochs):

    # TRAIN
    model.train()
    count, loss_train = 0, 0.0
    #loader contains dark matter cube, dm subhalo cube, subhalo + galaxy cube, dm + galaxy mask, and coords. 
    for maps,  params_true in train_loader: #currently ignoring subhalo structure. 
        maps = maps.to(device).float()
        params_true = params_true.to(device).float()
        # Forward Pass
        optimizer.zero_grad()
        params_pred = model(maps)
        loss = criterion(params_pred, params_true)
        loss_train += loss.cpu().detach().numpy()
        
        # Backward Prop
        loss.backward()
        optimizer.step()
        
        count += 1
    loss_train /= count
    
    # VALID
    model.eval() 
    count, loss_valid = 0, 0.0
    with torch.no_grad():
        for maps, params_true  in val_loader:
            
            maps = maps.to(device).float()
            params_true = params_true.to(device).float()
            params_pred = model(maps)
            error    = criterion(params_pred, params_true)   
            loss_valid += error.cpu().numpy()
            count += 1
    loss_valid /= count
     
        
    print('%03d %.4e %.4e'%(epoch, loss_train, loss_valid))
    # update learning rate
    scheduler.step(loss_valid)
    # save results to file
    f = open(fout, 'a')
    f.write('%d %.4e %.4e %.4e\n'%(epoch, loss_train, loss_valid, loss_test))
    f.close()

In [17]:
import torch

In [18]:
X_in = torch.Tensor(X_train[0:10]).resize(10,3,480,640)

/Users/noahkasmanoff/anaconda3/lib/python3.7/site-packages/torch/tensor.py:330: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Need some way to confirm the amount of samples I'm taking is the right amount. Frame rate data/train.mp4 is a video of driving containing 20400 frames. Video is shot at 20 fps.


In [19]:
out = model.forward(X_in)

Shape after conv 2  torch.Size([10, 32, 120, 160])


# Next steps:

Tell image diff it belongs in a training and validation dataset, make it split in a time based way though! Only sample last part or something.


Make a model to use, this will be MSE on a single value, start with some vanilla CNN. 